In [2]:
import numpy as np
import pandas as pd
import glob
import random

In [116]:
from geopy import distance

coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)

print (distance.geodesic(coords_1, coords_2).km)

279.35290160430094


In [120]:
trippath =r'E:\Data\Trainingdata\tripsample'
# trippath = r'E:\Data\test\trip'
Trip_file = glob.glob(trippath + "/*.txt")
logpath =r'E:\Data\Trainingdata\logsample'
# logpath = r'E:\Data\test\log'
Log_file = glob.glob(logpath + "/*.txt")

In [121]:
lon_range = [103.6, 104.042]
lat_range = [1.238, 1.48]

In [157]:
def preprocess(df):
    print("df shape before Lat Lon process:")
    print(df.shape)

    ## Lat Lon ##
    df = df[(df['Lat'] >= lat_range[0]) & (df['Lat'] <= lat_range[1])]
    df = df[(df['Lon'] >= lon_range[0]) & (df['Lat'] <= lon_range[1])]
    print("df shape after Lat Lon process:")
    print(df.shape)    
    
    ## Time ##
    df['Time'] = pd.to_datetime(df['Time'], format="%Y-%m-%d %X")
    df.sort_values(by='Time', inplace=True, ascending=True)

    df['Timedif'] = df['Time'] - df['Time'].shift(1)
    df['Timedif'] = df['Timedif'].astype(np.int64) // 10 ** 9.
    df = df.drop(['Time'], axis=1)
    df = df.reset_index(drop=True)
    df.iloc[0]['Timedif'] = 0
    df.iloc[0]['Timedif'] = np.mean(df['Timedif'])

    
    ## Distance ##
    coords = (df['Lat'], df['Lon'])
#     print(coords)
    lat = coords[0]
    lon = coords[1]
    distdif = []
    for i in range(0, len(lat) - 1):
#         print(distance.geodesic((lat[i], lon[i]), (lat[i + 1], lon[i + 1])).km)
        distdif.append(distance.geodesic((lat[i], lon[i]), (lat[i + 1], lon[i + 1])).km)
    mean = np.mean(distdif)
    distdif.insert(0, mean)
    df['Distdif'] = distdif
#     print("dist mean")
#     print(mean)
#     print(df['Distdif'])
    
    
    ## Speed ##
    speed = []
    timedif = df['Timedif']
#     print(source)
    for i in range(1, len(timedif)):
        speed.append(distdif[i] / timedif[i]) # km/s
#     print(np.shape(speed))
    mean = np.mean(speed)
    speed.insert(0, mean)
    df['Speed'] = speed
#     print("speed mean")
#     print(mean)
    
    ## Flag ## - 2 diff source will return 1, same source returns 0
    flag = []
    source = df['Source']
#     print(source)
    flag.append(0) # adding 0 to the first value
    for i in range(len(source) - 1):
        if source[i] != source[i + 1]:
            flag.append(1)
        else:
            flag.append(0)
#     print(np.shape(flag))
    df['Flag'] = flag    
    
    ## Drop first row ##
    df = df.drop([0])
    print("df shape final:")
    print(df.shape)
    
#     out = df.values.reshape(1, 4193) ## edit as necessary
#     print("out shape:")
#     print(out.shape)
    return df

In [123]:
header = ['Lat', 'Lon', 'Source', 'Timedif', 'Distdif', 'Speed', 'Flag']
# records = np.empty(shape=[0, 7]) ## edit as necessary
# labels = np.empty(shape=[0])
records = pd.DataFrame(columns=header)
labels = pd.DataFrame(columns=['Label'])
speed0 = pd.DataFrame(columns=header)
speed1 = pd.DataFrame(columns=header)

In [124]:
index = 0
while index < len(Trip_file):
    ## same ##
    df1 = pd.read_csv(Trip_file[index], usecols=[1, 2, 3, 4], sep=' ')
    df2 = pd.read_csv(Log_file[index], usecols=[1,2,3,4], sep=' ')
    out = preprocess(pd.concat([df1, df2]))
    records = pd.concat([records, out])
    labels = pd.concat([labels, pd.DataFrame(data=[1])])
    speed1 = pd.concat([speed1, out])
#     records = np.row_stack((records, out))
#     labels = np.append(labels, 1)
    
    ## different ##
    rand = random.randint(0, len(Trip_file) - 1)
    while rand == index:
        rand = random.randint(0, len(Trip_file) - 1)
    df3 = pd.read_csv(Log_file[rand], usecols=[1,2,3,4], sep=' ')
    out = preprocess(pd.concat([df1, df3]))
    records = pd.concat([records, out])
    labels = pd.concat([labels, pd.DataFrame(data=[0])])
    speed0 = pd.concat([speed1, out])
#     records = np.row_stack((records, out))
#     labels = np.append(labels, 0)
    index = index + 1

df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(598, 4)
df shape final:
(597, 7)
df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(599, 4)


C:\Users\lihao.A-I2R\AppData\Local\Continuum\anaconda3\envs\conda3.7\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\lihao.A-I2R\AppData\Local\Continuum\anaconda3\envs\conda3.7\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in double_scalars


df shape final:
(598, 7)
df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(597, 4)
df shape final:
(596, 7)
df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(597, 4)


C:\Users\lihao.A-I2R\AppData\Local\Continuum\anaconda3\envs\conda3.7\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\lihao.A-I2R\AppData\Local\Continuum\anaconda3\envs\conda3.7\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: divide by zero encountered in double_scalars


df shape final:
(596, 7)
df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(600, 4)
df shape final:
(599, 7)
df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(600, 4)
df shape final:
(599, 7)
df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(599, 4)
df shape final:
(598, 7)
df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(600, 4)
df shape final:
(599, 7)
df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(600, 4)
df shape final:
(599, 7)
df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(600, 4)
df shape final:
(599, 7)
df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(600, 4)
df shape final:
(599, 7)
df shape before Lat Lon process:

...

df shape final:
(599, 7)
df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(599, 4)
df shape final:
(598, 7)
df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(599, 4)
df shape final:
(598, 7)
df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(600, 4)
df shape final:
(599, 7)
df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(600, 4)
df shape final:
(599, 7)
df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(600, 4)
df shape final:
(599, 7)
df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(600, 4)
df shape final:
(599, 7)
df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(590, 4)
df shape final:
(589, 7)
df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(595, 4)
df shape final:
(594, 7)
df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(600, 4)


C:\Users\lihao.A-I2R\AppData\Local\Continuum\anaconda3\envs\conda3.7\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: divide by zero encountered in double_scalars


df shape final:
(599, 7)
df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(600, 4)


C:\Users\lihao.A-I2R\AppData\Local\Continuum\anaconda3\envs\conda3.7\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: divide by zero encountered in double_scalars


df shape final:
(599, 7)


In [125]:
print(records)

         Lat        Lon  Source  Timedif    Distdif     Speed Flag
1    1.29442  103.82194     0.0    926.0   3.248303  0.003508    1
2    1.29146  103.85959     0.0    660.0   4.202883  0.006368    0
3    1.37800  103.85801     1.0   6749.0   9.570765  0.001418    1
4    1.28365  103.84243     0.0   3751.0  10.575842  0.002819    1
5    1.28813  103.83592     0.0    300.0   0.877673  0.002926    0
6    1.35007  103.85809     1.0   1174.0   7.279868  0.006201    1
7    1.39263  103.89544     0.0   5426.0   6.278885  0.001157    1
8    1.39198  103.90935     0.0    360.0   1.549668  0.004305    0
9    1.36826  103.89557     1.0   1919.0   3.038260  0.001583    1
10   1.31000  103.79540     0.0   4321.0  12.875367  0.002980    1
11   1.29046  103.84495     0.0    780.0   5.922644  0.007593    0
12   1.38531  103.84714     1.0   2432.0  10.490859  0.004314    1
13   1.34247  103.98245     1.0   6847.0  15.785908  0.002306    0
14   1.29980  103.84341     1.0   4793.0  16.177126  0.003375 

In [106]:
# speed1 = []
# speed0 = []


# # print(records[0].shape)
# # len(viewRecords)
# for i in range(len(records)):
#     if labels[i] == 1:
#         speed1.append(records[i])
#     else:
#         speed0.append(records[i])

# print(len(speed1), len(speed0))
# print(len(speed1[0]))

In [107]:
# ## Reshaping from x * 4193 to y * 7 ##
# speed0 = np.asarray(speed0)
# speed1 = np.asarray(speed1)
# # speed0.shape
# # speed0 = speed0.reshape((2995, 7)) ## edit as necessary
# speed0 = speed0.reshape((1789213, 7)) ## edit as necessary
# speed1 = speed1.reshape((1789213, 7)) ## edit as necessary
# print(speed1)

In [126]:
print(speed1)

         Lat        Lon  Source  Timedif    Distdif     Speed Flag
1    1.29442  103.82194     0.0    926.0   3.248303  0.003508    1
2    1.29146  103.85959     0.0    660.0   4.202883  0.006368    0
3    1.37800  103.85801     1.0   6749.0   9.570765  0.001418    1
4    1.28365  103.84243     0.0   3751.0  10.575842  0.002819    1
5    1.28813  103.83592     0.0    300.0   0.877673  0.002926    0
6    1.35007  103.85809     1.0   1174.0   7.279868  0.006201    1
7    1.39263  103.89544     0.0   5426.0   6.278885  0.001157    1
8    1.39198  103.90935     0.0    360.0   1.549668  0.004305    0
9    1.36826  103.89557     1.0   1919.0   3.038260  0.001583    1
10   1.31000  103.79540     0.0   4321.0  12.875367  0.002980    1
11   1.29046  103.84495     0.0    780.0   5.922644  0.007593    0
12   1.38531  103.84714     1.0   2432.0  10.490859  0.004314    1
13   1.34247  103.98245     1.0   6847.0  15.785908  0.002306    0
14   1.29980  103.84341     1.0   4793.0  16.177126  0.003375 

In [80]:
# ## Adding header to the dataframes ##
# s0 = pd.DataFrame(speed0, dtype=float)
# s1 = pd.DataFrame(speed1, dtype=float)
# s0.columns = header
# s1.columns = header
# print(s1)

             Lat        Lon  Source  Timedif    Distdif     Speed  Flag
0        1.29442  103.82194     0.0    926.0   3.248303  0.003508   1.0
1        1.29146  103.85959     0.0    660.0   4.202883  0.006368   0.0
2        1.37800  103.85801     1.0   6749.0   9.570765  0.001418   1.0
3        1.28365  103.84243     0.0   3751.0  10.575842  0.002819   1.0
4        1.28813  103.83592     0.0    300.0   0.877673  0.002926   0.0
5        1.35007  103.85809     1.0   1174.0   7.279868  0.006201   1.0
6        1.39263  103.89544     0.0   5426.0   6.278885  0.001157   1.0
7        1.39198  103.90935     0.0    360.0   1.549668  0.004305   0.0
8        1.36826  103.89557     1.0   1919.0   3.038260  0.001583   1.0
9        1.31000  103.79540     0.0   4321.0  12.875367  0.002980   1.0
10       1.29046  103.84495     0.0    780.0   5.922644  0.007593   0.0
11       1.38531  103.84714     1.0   2432.0  10.490859  0.004314   1.0
12       1.34247  103.98245     1.0   6847.0  15.785908  0.00230

In [130]:
## Check if any index has a null value ##
print(speed1.notnull().values.all(), speed0.notnull().values.all())

False False


In [131]:
## Handling inf/-inf/NaN/null values + values at the extremities ##
print(speed1.shape, speed0.shape)
subset = ["Speed", "Distdif", "Timedif", "Lat", "Lon"]
speed1 = speed1.replace([np.inf, -np.inf], np.nan).dropna(subset=subset, how="any")
speed0 = speed0.replace([np.inf, -np.inf], np.nan).dropna(subset=subset, how="any")
print(speed1.shape, speed0.shape)

(1784094, 7) (1783991, 7)
(1778932, 7) (1778938, 7)


In [132]:
np.savetxt('records.txt', records, delimiter=',')
np.savetxt('labels.txt', labels, delimiter=',')
np.savetxt('datalabel0.txt', speed0.values, delimiter=',')
np.savetxt('datalabel1.txt', speed1.values, delimiter=',')

In [158]:
file = pd.read_csv('E:\Data\Trainingdata\logsample\SH1033G.txt', usecols=[1,2,3,4], sep=' ')
print(file)

         Lat        Lon  Source                 Time
0    1.35078  103.98562     1.0  2008-08-24 00:00:13
1    1.37079  103.86416     1.0  2008-08-24 01:51:59
2    1.35688  103.95028     1.0  2008-08-24 08:11:57
3    1.36031  103.89315     1.0  2008-08-24 09:57:36
4    1.31272  103.88053     1.0  2008-08-24 12:18:57
5    1.30300  103.90503     1.0  2008-08-24 14:02:51
6    1.31978  103.85379     1.0  2008-08-24 15:57:09
7    1.29056  103.83487     1.0  2008-08-24 18:13:55
8    1.37415  103.84595     1.0  2008-08-24 20:20:44
9    1.43412  103.76553     1.0  2008-08-24 22:05:52
10   1.31727  103.96425     1.0  2008-08-24 23:36:26
11   1.31279  103.85986     1.0  2008-08-19 01:04:39
12   1.37078  103.86416     1.0  2008-08-19 02:41:18
13   1.35697  103.93630     1.0  2008-08-19 07:31:06
14   1.32236  103.75045     1.0  2008-08-19 09:32:21
15   1.33466  103.75849     1.0  2008-08-19 12:32:42
16   1.35121  103.88429     1.0  2008-08-19 14:25:27
17   1.38378  103.90096     1.0  2008-08-19 17

In [159]:
file = preprocess(file)
print(file)

df shape before Lat Lon process:
(300, 4)
df shape after Lat Lon process:
(299, 4)
df shape final:
(298, 7)
         Lat        Lon  Source  Timedif    Distdif     Speed  Flag
1    1.37800  103.85801     1.0   8335.0  10.618413  0.001274     0
2    1.35007  103.85809     1.0   5225.0   3.088370  0.000591     0
3    1.36826  103.89557     1.0   7705.0   4.630717  0.000601     0
4    1.38531  103.84714     1.0   7533.0   5.709883  0.000758     0
5    1.34247  103.98245     1.0   6847.0  15.785908  0.002306     0
6    1.29980  103.84341     1.0   4793.0  16.177126  0.003375     0
7    1.31763  103.88728     1.0   9459.0   5.265364  0.000557     0
8    1.29534  103.86066     1.0   5332.0   3.853774  0.000723     0
9    1.36354  103.84746     1.0   3881.0   7.682957  0.001980     0
10   1.37074  103.86419     1.0   4833.0   2.024924  0.000419     0
11   1.27577  103.85909     1.0  23094.0  10.516622  0.000455     0
12   1.29197  103.84319     1.0  16278.0   2.517949  0.000155     0
13   1.3

In [16]:
speed0 = pd.read_csv(r"C:\Users\lihao.A-I2R\Desktop\Stanley\data\datalabel0.txt", sep=',', header=None)
speed1 = pd.read_csv(r"C:\Users\lihao.A-I2R\Desktop\Stanley\data\datalabel1.txt", sep=',', header=None)
speed0

,0,1,2,3,4,5,6
0,1.31230,103.80396,1.0,3132.0,3.779294,0.001207,0.0
1,1.29442,103.82194,0.0,3942.0,2.812984,0.000714,1.0
2,1.29146,103.85959,0.0,660.0,4.202883,0.006368,0.0
3,1.28025,103.82692,1.0,519.0,3.841383,0.007402,1.0
4,1.28796,103.82008,1.0,4926.0,1.142930,0.000232,0.0
5,1.28365,103.84243,0.0,5055.0,2.532613,0.000501,1.0
6,1.28813,103.83592,0.0,300.0,0.877673,0.002926,0.0
7,1.39263,103.89544,0.0,6600.0,13.319019,0.002018,0.0
8,1.39198,103.90935,0.0,360.0,1.549668,0.004305,0.0
9,1.35504,103.84551,1.0,203.0,8.195103,0.040370,1.0


In [17]:
header = ['Lat', 'Lon', 'Source', 'Timedif', 'Distdif', 'Speed', 'Flag']
speed0.columns = header
speed1.columns = header
speed1

,Lat,Lon,Source,Timedif,Distdif,Speed,Flag
0,1.29442,103.82194,0.0,926.0,3.248303,0.003508,1.0
1,1.29146,103.85959,0.0,660.0,4.202883,0.006368,0.0
2,1.37800,103.85801,1.0,6749.0,9.570765,0.001418,1.0
3,1.28365,103.84243,0.0,3751.0,10.575842,0.002819,1.0
4,1.28813,103.83592,0.0,300.0,0.877673,0.002926,0.0
5,1.35007,103.85809,1.0,1174.0,7.279868,0.006201,1.0
6,1.39263,103.89544,0.0,5426.0,6.278885,0.001157,1.0
7,1.39198,103.90935,0.0,360.0,1.549668,0.004305,0.0
8,1.36826,103.89557,1.0,1919.0,3.038260,0.001583,1.0
9,1.31000,103.79540,0.0,4321.0,12.875367,0.002980,1.0


In [35]:
def arrange_by_flag(df):
    flag0 = []
    flag1 = []
    flag = df["Flag"]
    
    for i in range(len(flag)):
        if flag[i] == 1:
#             print(df.iloc[i, :])
#             flag1 = pd.concat([flag1, df.iloc[i, :]])
            flag1.append(df.iloc[i, :])
        else:
#             print(df.iloc[i, :])
#             flag0 = pd.concat([flag0, df.iloc[i, :]])
            flag0.append(df.iloc[i, :])

    return flag0, flag1

In [36]:
flag0 = pd.DataFrame(columns=header)
flag1 = pd.DataFrame(columns=header)
s0f0 = pd.DataFrame(columns=header)
s0f1 = pd.DataFrame(columns=header)
s1f0 = pd.DataFrame(columns=header)
s1f1 = pd.DataFrame(columns=header)

flags = arrange_by_flag(speed0)

flag0 = flag0.append(flags[0])
flag1 = flag1.append(flags[1])
s0f0 = s0f0.append(flags[0])
s0f1 = s0f1.append(flags[1])

flags = arrange_by_flag(speed1)

flag0 = flag0.append(flags[0])
flag1 = flag1.append(flags[1])
s1f0 = s1f0.append(flags[0])
s1f1 = s1f1.append(flags[1])

In [40]:
print(s1f1.shape, s0f1.shape, flag1.shape)
print(s1f0.shape, s0f0.shape, flag0.shape)

(1189144, 7) (910442, 7) (2099586, 7)
(589788, 7) (868496, 7) (1458284, 7)


In [41]:
np.savetxt('flag1.txt', flag1, delimiter=',')
np.savetxt('flag0.txt', flag0, delimiter=',')
np.savetxt('s1f1.txt', s1f1, delimiter=',')
np.savetxt('s1f0.txt', s1f0, delimiter=',')
np.savetxt('s0f1.txt', s0f1, delimiter=',')
np.savetxt('s0f0.txt', s0f0, delimiter=',')